In [1]:
import numpy as np 
import cupy as cp
import tiffile as tiff
import matplotlib.pyplot as plt

## Cylinder mask

In [2]:
def circle_mask(size, radius):
    mask = np.zeros([size, size, size], dtype = np.uint8)
    X, Y = np.ogrid[:size, :size]
    dist_from_center = np.sqrt((X - size // 2)**2 + (Y - size // 2)**2, dtype = np.float32)
    mask = dist_from_center <= radius
    return mask

## Sphere generator

In [3]:
def circle(size, center, radius):
    mask = np.zeros([size, size, size], dtype = np.uint8)
    X, Y, Z = np.ogrid[:size, :size, :size]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2 + (Z-center[2])**2, dtype = np.float32)
    mask = dist_from_center<= radius
    return mask 

def circle_cuda(size, center, radius):
    mask = cp.zeros([size, size, size], dtype = cp.uint8)
    X, Y, Z = cp.ogrid[:size, :size, :size]
    dist_from_center = cp.sqrt((X - center[0])**2 + (Y-center[1])**2 + (Z-center[2])**2, dtype = cp.float32)
    mask = dist_from_center<= radius
    return mask

## 3D spheres packing generation 

In [4]:
# number of iterations
N = 100

# box size
size = 512

# maximum value of sphere radius variation (R_min + rand(0, 1)*delta)
delta = 7 

# minimal sphere radius
r_min = 20

box = cp.zeros([size, size, size], dtype = cp.uint8)

x0 = cp.random.randint(0, size, 1)
y0 = cp.random.randint(0, size, 1)
z0 = cp.random.randint(0, size, 1)
r0 = r_min + cp.random.random() * delta

cent = [[x0, y0, z0]]
radius = [r0]

box += circle_cuda(size = size, center = [x0, y0, z0], radius = r0)

# iterator
i = 0

while (i < N):
    
    #print(i)
    
    xi = cp.random.randint(0, size, 1)
    yi = cp.random.randint(0, size, 1)
    zi = cp.random.randint(0, size, 1)
    
    ri = r_min + cp.random.random() * delta
    
    circle_i = circle_cuda(size = size, center = [xi, yi, zi], radius = ri)
    
    if (len((box + circle_i)[cp.where((box + circle_i) == 2)]) == 0):
        
        box += circle_i
        cent.append([xi, yi, zi])
        radius.append(ri)
        
    i+=1

## Move spheres

In [ ]:
# number of iterations 
N = 1000 

# maximal step size in pixels  
step = 10

# convert list of coordinates to numpy array
arr_cent = cp.array(cent)
arr_cent = cp.asnumpy(arr_cent)

# save spheres radiuses
np.save('move_3D/radius.npy', radius)

# iterator
i = 0 

while (i < N):
    
    print(i)
    
    box = cp.zeros([size, size, size], dtype = cp.uint8)
    
    x = cp.asarray(arr_cent)[:, 0, 0]
    y = cp.asarray(arr_cent)[:, 1, 0]
    z = cp.asarray(arr_cent)[:, 2, 0]

    idx = int(np.random.randint(0, len(arr_cent), 1))
    
    dx = (cp.random.random() - cp.random.random()) * cp.uint8(step)  
    dy = (cp.random.random() - cp.random.random()) * cp.uint8(step)  
    dz = (cp.random.random() - cp.random.random()) * cp.uint8(step)  
    
    x[idx] = x[idx] + dx
    y[idx] = y[idx] + dy
    z[idx] = z[idx] + dz
   
    for j in range(len(cent)):
        circle_i = circle_cuda(size = size, center = [x[j], y[j], z[j]], radius = radius[j])
        box += circle_i
    
    if (len(box[cp.where(box == 2)]) == 0): 
          
        for k in range(len(cent)):
            arr_cent[k, 0] = x.get()[k]
            arr_cent[k, 1] = y.get()[k]
            arr_cent[k, 2] = z.get()[k]
            
        # save spheres packing 
        tiff.imwrite('move_3D/frame_' + str(i) + '.tiff', box.get() * 255)
        # save centers coordinates 
        np.save('move_3D/coord_' + str(i) + '.npy', arr_cent)
        
    i+=1